In [6]:
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, f1_score
import numpy as np
import pandas as pd
from core import semcor_bert_pipeline
from core.metrics import *

In [10]:
shared_metrics = {}
shared_words = ['foot.n', 'plane.n', 'model.n', 'table.n', 'degree.n',]
fmt_sense_12 = lambda w: [w + '.01', w + '.02']
shared_senses = [fmt_sense_12(w) for w in shared_words]
shared_senses[-1] = ['degree.n.01', 'academic_degree.n.01']
shared_senses[1] = ['airplane.n.01', 'plane.n.02']
for w_s in zip(shared_words, shared_senses):
    model_data = binary_logistic(w_s[0], w_s[1])
    weight_values, weight_indices = nonzero_weights(model_data['model'])
    f_scores, accuracies, wrong_indices = k_fold_cv(model_data['data'], model_data['transformed_labels'])
    shared_metrics[w_s[0]] = {'senses': w_s[1], 'data': model_data, 'weights': weight_values,
                    'weight_indices': weight_indices, 'f1_kfold': f_scores, 'acc_kfold': accuracies,
                             'incorrect_indices': wrong_indices}
    

/anaconda3/envs/bert/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)
/anaconda3/envs/bert/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)
/anaconda3/envs/bert/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)
/anaconda3/envs/bert/lib/python3.7/site-packages/sklearn/metr

In [11]:
shared_stats = []
incorrect = []
for k in shared_metrics:
    word_results = shared_metrics[k]
    weights = word_results['weights']
    incorrect.append(misclassified_sentences(word_results['data'], word_results['incorrect_indices']))
    shared_stats.append({'word_type': k, 'avg_f1': np.mean(word_results['f1_kfold']), 'avg_acc': np.mean(word_results['acc_kfold']),
    'pct_nonzero_weights': len(weights) / 768, 'max_wt': max(weights), 'min_wt': min(weights), 'mean_nonzero_wt': np.mean(weights), 'sd_nonzero_wt': np.std(weights)})
pd.DataFrame(shared_stats)

,word_type,avg_f1,avg_acc,pct_nonzero_weights,max_wt,min_wt,mean_nonzero_wt,sd_nonzero_wt
0,foot.n,0.600000,0.973333,0.031250,0.221365,-0.229638,0.028897,0.132391
1,plane.n,0.333333,0.940000,0.015625,0.491047,-0.264988,0.054570,0.223150
2,model.n,0.209091,0.834545,0.037760,0.273954,-0.291383,0.020789,0.136815
3,table.n,0.792000,0.993333,0.033854,0.476585,-0.254916,0.013813,0.151199
4,degree.n,0.800000,1.000000,0.018229,0.456815,-0.227535,0.001083,0.166228


Misclassified senses (Only binary classification, 3/16 senses for math plane were misclassified)

In [12]:
pd.set_option('display.max_colwidth', 500)
pd.concat(incorrect)

,true_label,sentences
0,foot.n.02,I cannot make sense out of the figures for post maturity growth ; at best the annual increase appears to be a matter of inches rather than feet .
1,foot.n.02,He rejects dimensions of 70 feet and more .
2,foot.n.02,"In contrast , Ditmars recorded the average length of seventy-two young of a 19 - foot female as 38 inches , and four young were born in London at a length of 35 or 36 inches and a weight of from 14 to 16 ounces ."
3,foot.n.02,"Hans Schweizer had one that increased from 19 - 1 2 inches to 5 feet 3 inches in five years , and J . J . Quelch records a growth of from less than 4 feet to nearly 10 in about six years ."
0,plane.n.02,"If a is the major axis of an ellipsoid and b and c are the other two axes , the radius of curvature in the ab plane at the end of the axis is * * f , and the difference in pressure along the a and b axes is * * f ."
1,plane.n.02,"Also , planetary gravitational attraction increases the dust concentration near the plane of the ecliptic as the sun is approached ."
2,plane.n.02,We will refer to the plane of C and **f as the C-plane and to the plane of the graph as the f-plane .
0,model.n.01,The Glazer-Fine Arts edition ( Concert-Disc ) is a model of lucidity and organization .
1,model.n.01,"I asked Quasimodo recently how he accomplished this , and he replied that he had painted his model `` a beautiful shade of red and then had her breathe on the canvas '' , which was his typical tongue-in-cheek way of chiding me for my lack of sensitivity ."
2,model.n.01,"The information provided by the electron paramagnetic effects is then discussed , and finally the nuclear effects are interpreted in terms of various motional modified models of the **f bond in **f ."


I thought this might be useful to see if weights at similar positions were used.

In [13]:
[(k, shared_metrics[k]['weight_indices']) for k in shared_words]

[('foot.n',
  array([ 13,  29,  40,  70,  89, 184, 191, 217, 231, 278, 287, 304, 308,
         332, 493, 518, 528, 532, 547, 637, 664, 693, 709, 730])),
 ('plane.n',
  array([105, 157, 254, 286, 308, 324, 411, 413, 448, 513, 539, 619])),
 ('model.n',
  array([ 18,  49,  68,  74,  79, 101, 114, 242, 254, 259, 282, 289, 340,
         364, 450, 473, 477, 509, 520, 523, 525, 527, 552, 574, 586, 597,
         618, 739, 749])),
 ('table.n',
  array([ 37,  67,  94, 136, 143, 188, 213, 228, 252, 286, 298, 317, 332,
         334, 363, 376, 393, 445, 481, 487, 552, 661, 666, 680, 705, 753])),
 ('degree.n',
  array([ 15,  22, 220, 262, 328, 350, 432, 523, 537, 541, 544, 620, 622,
         724]))]